# Use O2C_Template Customized Notebook Template

In [1]:
import random
from faker import Faker
import pandas as pd

In [5]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [6]:
fake = Faker('en_US')

# Customer Data Setup

In [7]:
# Generate Customers
def generate_customers(num_customers=100):
    customers = []
    for _ in range(num_customers):
        customer = {
            'CustomerID': fake.uuid4(),
            'Name': fake.company(),
            'CompanyType':fake.company_suffix(),
            'ContactDetails': fake.phone_number(),
            'EmailDetails': fake.company_email(),
            'AddressDetails':fake.address(),
            'AdminDetails':fake.administrative_unit(),
            'CreditLimit': random.randint(10000, 100000),
            'CreditRating': random.choice(['very poor', 'fair', 'good', 'very good','exceptional'])
        }
        customers.append(customer)
    return customers

In [8]:
# Generate Data
customers = generate_customers()

In [9]:
df_customers = pd.DataFrame(customers)

In [10]:
df_customers.head()

,CustomerID,Name,CompanyType,ContactDetails,EmailDetails,AddressDetails,AdminDetails,CreditLimit,CreditRating
0,160ca858-a03e-4595-8519-e9bb1620fd27,Klein-Cordova,Ltd,+1-562-488-1896x4491,fstout@white.com,"5596 Wendy Mission Suite 873\nEast Julieport, ...",Florida,23347,very good
1,b95cb40b-2d81-428f-92ac-018aa7ec8770,Mann Inc,Ltd,113.867.1672x291,fieldsaustin@turner-floyd.info,"4057 Samuel Crest\nGarrettview, OH 42172",New Hampshire,32797,fair
2,f408a3a9-f257-4ec4-9bcb-fcdd88ea4d01,"Reeves, Ingram and Burnett",and Sons,(746)331-0507x64212,daniel33@fischer-downs.com,72043 Sullivan Gardens Suite 364\nLindachester...,Rhode Island,12506,exceptional
3,ac324cdd-48bc-422a-9a5b-934b2c906d27,Miller PLC,Inc,455-254-3815x4095,jasonturner@nunez.com,"050 Cook Shore Apt. 885\nPort Stacie, IA 54819",Utah,96504,very poor
4,895ce235-aa50-4321-b6ab-f98f06c42b10,"Schneider, Lane and Ramos",PLC,300-649-1158x3210,stephanierobbins@king-payne.com,"025 Jocelyn Roads\nPort Tammyhaven, MO 59117",Wyoming,28488,very poor


In [12]:
df_customers.columns = ['CustomerID', 'Name', 'CompanyType', 'ContactDetails', 'EmailDetails',
       'AddressDetails', 'AdminDetails', 'CreditLimit', 'CreditRating']

Index(['CustomerID', 'Name', 'CompanyType', 'ContactDetails', 'EmailDetails',
       'AddressDetails', 'AdminDetails', 'CreditLimit', 'CreditRating'],
      dtype='object')

In [25]:
df_train_sf=my_session.createDataFrame(
        df_customers.values.tolist(),
        schema=df_customers.columns.tolist())
df_train_sf.write.mode("overwrite").save_as_table("FDC_HORIZONTAL.O2C_SCHEMA.BRONZE_CUSTOMER_DATA")

# Product Data setup

In [20]:
df_product = pd.read_csv('product_files.csv')

In [21]:
df_product.head()

,ProductCategory,ProductType,Quantity,UnitPrice
0,Office Supplies,Laser Printer,100,425
1,Office Supplies,Ergonomic Chair,1000,299
2,Office Supplies,Standing Desk,1000,420
3,Office Supplies,Wireless Keyboard,1000,78
4,Office Supplies,Office Stationery Set,1000,15


In [16]:
def generate_products(data):
    data['ProductID'] = ''
    # Iterate over rows and columns
    for index, row in data.iterrows():
        data.at[index, 'ProductID'] = fake.uuid4()
    return data

In [22]:
df_product = generate_products(df_product)

In [26]:
df_train_sf=my_session.createDataFrame(
        df_product.values.tolist(),
        schema=df_product.columns.tolist())
df_train_sf.write.mode("overwrite").save_as_table("FDC_HORIZONTAL.O2C_SCHEMA.BRONZE_PRODUCT_DATA")

# Order Items Data Set up

In [ ]:
# Generate Orders and Order Items
def generate_orders_and_items(customers, products, num_orders=100000):
    orders = []
    order_items = []
    shipments = []
    invoices = []
    payments = []
    statuses = ['Fulfilled', 'Delayed Fulfillment', 'Pending Payment']

    for _ in range(num_orders):
        customer = random.choice(customers)
        order_status = random.choice(statuses)
        order_id = fake.uuid4()
        order_date = fake.date_this_year()
        total_amount = 0

        order = {
            'OrderID': order_id,
            'CustomerID': customer['CustomerID'],
            'OrderDate': order_date,
            'OrderStatus': order_status,
            'TotalAmount': total_amount  # Will be updated later
        }
        orders.append(order)

        num_items = random.randint(1, 5)
        for _ in range(num_items):
            product = random.choice(products)
            quantity = random.randint(1, 10)
            total_price = product['Price'] * quantity
            total_amount += total_price

            order_item = {
                'OrderItemID': fake.uuid4(),
                'OrderID': order_id,
                'ProductID': product['ProductID'],
                'Quantity': quantity,
                'UnitPrice': product['Price'],
                'TotalPrice': total_price
            }
            order_items.append(order_item)

        # Update the total amount for the order
        order['TotalAmount'] = total_amount

        shipment = {
            'ShipmentID': fake.uuid4(),
            'OrderID': order_id,
            'ShipmentDate': fake.date_this_year(),
            'Carrier': fake.company(),
            'TrackingNumber': fake.uuid4(),
            'ShipmentStatus': 'Shipped' if order_status == 'Fulfilled' else 'Pending'
        }
        shipments.append(shipment)

        invoice = {
            'InvoiceID': fake.uuid4(),
            'OrderID': order_id,
            'InvoiceDate': fake.date_this_year(),
            'DueDate': fake.date_this_year(),
            'TotalAmount': total_amount,
            'PaymentStatus': 'Paid' if order_status == 'Fulfilled' else 'Pending'
        }
        invoices.append(invoice)

        if order_status == 'Fulfilled':
            payment = {
                'PaymentID': fake.uuid4(),
                'InvoiceID': invoice['InvoiceID'],
                'PaymentDate': fake.date_this_year(),
                'PaymentAmount': total_amount,
                'PaymentMethod': random.choice(['Credit Card', 'Bank Transfer', 'Cash'])
            }
            payments.append(payment)

    return orders, order_items, shipments, invoices, payments

In [ ]:
orders, order_items, shipments, invoices, payments = generate_orders_and_items(customers, products)

In [ ]:
# Convert to DataFrames for easy manipulation


df_orders = pd.DataFrame(orders)
df_order_items = pd.DataFrame(order_items)
df_shipments = pd.DataFrame(shipments)
df_invoices = pd.DataFrame(invoices)
df_payments = pd.DataFrame(payments)

In [1]:
orders

NameError: name 'orders' is not defined